In [483]:
import warnings
warnings.simplefilter('ignore')

In [484]:
from urllib import request as req

data_dir = '/Users/ssv/newprolab/project01'
filename = 'gender_age_dataset.txt'
file_path = '/'.join([data_dir, filename])
url = 'http://data.newprolab.com/data-newprolab-com/project01/' + filename

# req.urlretrieve(url, file_path)

In [485]:
import pandas as pd
import json
from urllib.parse import urlparse
from datetime import datetime

from tqdm import tqdm_notebook

In [486]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 4000)

In [487]:
df = pd.read_csv(file_path, sep='\t')

### Оцифровка значений пола и возраста

In [488]:
ages = {'18-24': 1, '25-34': 2, '35-44': 3, '45-54': 4, '>=55': 5}
df["age"] = df["age"].apply(lambda x: ages.get(x, '-'))

In [489]:
genders = {'M': 1, 'F': 0}
df["gender"] = df["gender"].apply(lambda x: genders.get(x, '-'))


### Формирование набора данных с ключевыми словами посещенных сайтов

#### Загружаем из файлов типа df_med_keywords_0000_0999.pickle по мере поступления. Скрапинг сайтов идет очень медленно

In [490]:
import pickle

In [491]:
with open('data/keywords_999.pickle', 'rb') as file:
    df_med_keywords_0000_0999 = pickle.load(file)

In [492]:
with open('data/keywords_2000_6999.pickle', 'rb') as file:
    df_med_keywords_2000_6999 = pickle.load(file)

In [493]:
with open('data/df_med_keywords_non_age.pickle', 'rb') as file:
    df_for_pred = pickle.load(file)

#### Удаляем лишние столбцы в наборе данных после скрапинга

In [494]:
df_med_keywords_0000_0999 = df_med_keywords_0000_0999.drop('url_netloc', axis=1)

In [495]:
df_med_keywords_2000_6999 = df_med_keywords_2000_6999.drop('url_netloc', axis=1)

In [496]:
df_for_pred = df_for_pred.drop(['url_netloc', 'age'], axis=1)

#### Соединяем загруженные таблицы

In [497]:
df_k = pd.merge(df_med_keywords_0000_0999, pd.merge(df_med_keywords_2000_6999, df_for_pred, how='left', on='uid'),
              how='left', on='uid')

#### Сводим ключевые слова в один столбец

In [498]:
df['keywords'] = ''
for index, row in tqdm_notebook(df_k.iterrows()):
    if index < 1000:
        df_k['raw_keywords'].loc[index] = df_k['raw_keywords'].iloc[index]
    if index >= 2000 and index < 7000:
        df_k['raw_keywords'].loc[index] = df_k['raw_keywords_x'].iloc[index]
    if index > 36137:    
        df_k['raw_keywords'].loc[index] = df_k['raw_keywords_y'].iloc[index]
    

A Jupyter Widget

#### Левое последовательное соединение таблиц по uid. Удаляем лишние столбцы после соединения 

In [499]:
df_k = df_k.drop(['raw_keywords_x', 'raw_keywords_y'], axis=1)

In [500]:
df = pd.merge(df, df_k, how='left', on='uid')

In [501]:
df = df.drop(['user_json', 'keywords'], axis=1)

####  В результате - имеем набор df['gender', 'age', 'uid', 'raw_keywords']. Ключевые слова посещенных сайтов заполнены для строк 0 - 999; 2000 - 6999 и таргет: 36138 - 41137. В заполненных строках есть пустые значения (массив длиной 0)

1. Вытащим все ключевые слова для формирования массива стоп-слов
2. Разобьем датасет на два фолда: для тренировки (0000 - 36137) и предсказания (36138 - 41137)
3. Из фолдов удалим строки в которых нет ключевых слов сайтов
4. Сохраняем таргеты и тренируем модель

In [502]:
raw_words = list(df['raw_keywords'].copy())

In [503]:
for item in tqdm_notebook(reversed(raw_words)):
    if len(item) == 0:
        raw_words.remove(item)

A Jupyter Widget

In [504]:
import pymorphy2
m = pymorphy2.MorphAnalyzer()

In [505]:
import re
GROUPING_SPACE_REGEX = re.compile(r'([^\w]|[+])', re.UNICODE)

def simple_word_tokenize(text, _split=GROUPING_SPACE_REGEX.split):
    return [t for t in _split(text.lower()) if t and not t.isspace()]

In [506]:
def token_r(text):
    words = simple_word_tokenize(text)
    return [m.parse(x)[0].normal_form for x in words if len(x) >= 4]

In [507]:
raw_words = [' '.join((x)) for x in raw_words]

In [ ]:
clear_words = [' '.join(token_r(x)) for x in raw_words]

In [454]:
from sklearn.feature_extraction.text import CountVectorizer

In [455]:
cv = CountVectorizer()
matrix = cv.fit_transform(clear_words)

In [ ]:
# sorted(cv.vocabulary_.items())[1451:]

In [ ]:
# sorted(cv.vocabulary_.items(), reverse=True)[552:]

In [456]:
stop_ww = sorted(cv.vocabulary_.items(), reverse=True)[:552] + sorted(cv.vocabulary_.items())[:1451]

In [457]:
stop_w = [x[0] for x in stop_ww]

## Построение модели

1. Подготовка наборов данных
    - Разобьем датасет на два фолда: для тренировки (0000 - 36137) и предсказания (36138 - 41137)
    - Из фолдов удалим строки в которых нет ключевых слов сайтов

In [458]:
df_train = df.loc[0:36137]

In [459]:
df_train['raw_keywords'] = df_train['raw_keywords'].apply(lambda x: ' '.join(x))

In [460]:
df_train = df_train[df_train['raw_keywords'] != '']

In [480]:
df_train.columns

Index(['gender', 'age', 'uid', 'raw_keywords'], dtype='object')

In [462]:
df_train['raw_keywords']

0       Последние новости в Яндекс.Новостях: Актуальны...
1       EuroAvtoprokat.Ru предлагает лучшие цены на пр...
2       58994 кулинарных рецептов приготовления блюд н...
3       Сервис перевода татуировок надписей, живой про...
4       {description} {keywords} Общественно-политичес...
5       Cross Fire - захватывающий MMO-шутер со множес...
6       Объявления на Avito — подать объявления. объяв...
7       Купить технику Apple и аксессуары в фирменном ...
8       Cобытия и новости 24 часа в сутки: эксклюзивны...
9       www.tvcok.ru - новый подход к онлайн фильмам! ...
10      Объявления на Avito — подать объявления. объяв...
11      Подробные отзывы о косметике, лучшие уроки кра...
12      android.mobile-review.com — Лучший сайт про an...
13      'Новини в Україні и світі. Останні новини ТСН....
14      Сервис Playcast позволяет легко выразить свои ...
15      Биографии писателей, поэтов, художников, учены...
17      Rata news - новости туризма для профессионалов...
19      MSN Ро

#### Строим модель для обучения

In [481]:
target_age = df_train['age']
target_gender = df_train['gender']
df_train_uid = df_train['uid']
df_train = df_train.drop(['age', 'gender', 'uid'], axis=1)

In [482]:
cv = CountVectorizer(tokenizer=token_r, stop_words=stop_w)
matrix = cv.fit_transform(df_train['raw_keywords'])

KeyboardInterrupt: 

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    matrix, 
    target_age, 
    test_size=0.33,
    random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1)
rfc.fit(X_train, y_train) 
pred = rfc.predict(X_test)

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred[:,1])

In [ ]:
metrics.auc(fpr, tpr)

In [469]:
df_predict = df.loc[36138:]

In [470]:
df_predict_ALL_uid = df['uid'] # все UID до удаления тех, для которых НЕ будет выполняться прогноз !!!!

In [471]:
df_predict['raw_keywords'] = df_predict['raw_keywords'].apply(lambda x: ' '.join(x))

In [472]:
df_predict = df_predict[df_predict['raw_keywords'] != '']

#### Сохраняем столбцы age, gender - как будущий target для модели и столбец url_counts - как возможную фичу

In [477]:
target_age = df_predict['age']
target_gender = df_predict['gender']
df_predict_uid = df_predict['uid']
df_predict = df_predict.drop(['age', 'gender', 'uid'], axis=1)

In [ ]:
futures = 

In [251]:
df['raw_keywords']

0        [Последние новости в Яндекс.Новостях: Актуальн...
1        [EuroAvtoprokat.Ru предлагает лучшие цены на п...
2        [58994 кулинарных рецептов приготовления блюд ...
3        [Сервис перевода татуировок надписей, живой пр...
4        [{description}, {keywords}, Общественно-полити...
5        [Cross Fire - захватывающий MMO-шутер со множе...
6        [Объявления на Avito — подать объявления., объ...
7        [Купить технику Apple и аксессуары в фирменном...
8        [Cобытия и новости 24 часа в сутки: эксклюзивн...
9        [www.tvcok.ru - новый подход к онлайн фильмам!...
10       [Объявления на Avito — подать объявления., объ...
11       [Подробные отзывы о косметике, лучшие уроки кр...
12       [android.mobile-review.com — Лучший сайт про a...
13       ['Новини в Україні и світі. Останні новини ТСН...
14       [Сервис Playcast позволяет легко выразить свои...
15       [Биографии писателей, поэтов, художников, учен...
16                                                      

In [121]:
target_age = df['age']
target_gender = df['gender']
url_counts = df['url_counts']

In [122]:
df = df.drop(['age', 'gender', 'url_counts'], axis=1)

In [152]:
df.columns

Index(['gender', 'age', 'uid', 'raw_keywords'], dtype='object')

In [ ]:
%matplotlib inline
import seaborn as sns

In [ ]:
sns_plot = sns.pairplot(df[['url_counts', 'age']], size=4)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 16, 7
sns.distplot(df['url_counts'])

In [ ]:
df['url_counts'].value_counts().head(50).plot(kind='bar')

In [ ]:
rcParams['figure.figsize'] = 16, 7
sns.jointplot(df['age'], df['url_counts'], size=10)

In [ ]:
%lsmagic

In [ ]:
import pickle
with open('data/df_med_keywords_non_age.pickle', 'rb') as file:
    df_med_keywords_non_age = pickle.load(file)

In [ ]:
df_med_keywords_non_age['raw_keywords'].loc[36138:]

In [ ]:
41138 - 5000